<h2>Проект по SQL</h2>
Крупный сервис для чтения книг по подписке.<br>
Задача проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.
Эти данные помогут сформулировать ценностное предложение для нового продукта.

<h3>Описание данных</h3>

**Таблица books**

Содержит данные о книгах: 

    book_id — идентификатор книги;
    author_id — идентификатор автора;
    title — название книги;
    num_pages — количество страниц;
    publication_date — дата публикации книги;
    publisher_id — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:

    author_id — идентификатор автора;
    author — имя автора.

**Таблица publishers**

Содержит данные об издательствах:

    publisher_id — идентификатор издательства;
    publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

    rating_id — идентификатор оценки;
    book_id — идентификатор книги;
    username — имя пользователя, оставившего оценку;
    rating — оценка книги.
    
**Таблица reviews**

Содержит данные о пользовательских обзорах:

    review_id — идентификатор обзора;
    book_id — идентификатор книги;
    username — имя автора обзора;
    text — текст обзора.

**Задача**<br>
* Посчитать, сколько книг вышло после 1 января 2000 года;<br>
* Для каждой книги посчитать количество обзоров и среднюю оценку;<br>
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (так можно исключить из анализа брошюры;)<br>
* Определить автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками;)<br>
* Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.<br>

**План**<br>
* Цели исследования;<br>
* Исследование таблиц — вывести первые строки;<br>
* SQL-запросы для решения каждого задачи;<br>
* Вывести результат каждого запроса;<br>
* Выводы.<br>

In [3]:
# доступ к базе данных по сертификату СА.pem добавляется в папку проекта
# импорт библиотек

import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': '____________', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

In [ ]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

In [ ]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

In [ ]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))
pd.io.sql.read_sql(query, con = engine).info()

<h2>Запросы по задаче¶</h2>

**2.1. Посчитаем, сколько книг вышло после 1 января 2000 года**

In [ ]:
query = '''
SELECT
    COUNT(DISTINCT book_id) AS book_cnt
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

После 1 января 2000 года вышло 819 книг.

**2.2. Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [ ]:
query = '''
SELECT
    books.book_id AS id,
    books.title AS title,
    COUNT(DISTINCT reviews.review_id) AS cnt_reviews,
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM
    books
INNER JOIN reviews ON reviews.book_id = books.book_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY id
ORDER BY  avg_rating DESC, cnt_reviews DESC
'''
pd.io.sql.read_sql(query, con = engine)

**2.3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры**

In [ ]:
query = '''
SELECT
    publishers.publisher as publisher,
    COUNT(books.publisher_id) as cnt
FROM
    books
INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE
    books.num_pages > 50
GROUP BY publisher
ORDER BY cnt DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

Penguin Books выпустило больше всего книг объемом более 50 страниц.

**2.4. Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками**

In [ ]:
query = '''
SELECT 
    authors.author,
    ROUND(AVG(rating), 2) AS avg_rating,
    COUNT(rating_id) AS cnt_rating
FROM
    authors
INNER JOIN books ON authors.author_id = books.author_id
AND books.book_id IN (
                    SELECT
                        book_id
                    FROM
                        ratings
                    GROUP BY
                        book_id
                    HAVING COUNT(rating_id) >= 50
                    )
INNER JOIN ratings ON books.book_id = ratings.book_id
GROUP BY
    authors.author
ORDER BY AVG(rating) DESC
    
'''
pd.io.sql.read_sql(query, con = engine)

Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг.

**2.5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок***

In [ ]:
query = '''
SELECT
    'Cреднее количество обзоров от пользователей:',
    ROUND(AVG(cnt_reviews), 2) AS avg_cnt_reviews
FROM
    (SELECT
        (SELECT
            COUNT(*)
        FROM
            reviews
        WHERE
            username = ratings.username
        ) AS cnt_reviews
    FROM
        ratings
    GROUP BY
        username
    HAVING
    COUNT(*) >= 50
    ) AS SUBQ
'''
pd.io.sql.read_sql(query, con = engine)

* После 1 января 2000 года вышло 819 книг.
* Penguin Books выпустило больше всего книг объемом более 50 страниц.
* Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг.
* Среднее количество обзоров от пользователей 24